# PACKAGES
The first step is loading the packages that we will use:


In [ ]:
import pandas as pd #data manipulation
import numpy as np #data manipulation
from numpy import array
from numpy import hstack
from numpy.random import seed
seed(123)
import tensorflow as tf
import datetime
from datetime import datetime, timedelta
from sklearn import preprocessing #preprocessing
import math as math

import keras as keras #keras package for neural networks
from keras.models import Sequential
from keras.layers import Dense #dense layers
from keras.layers import LSTM #LSTM layers
from keras.layers import Dropout #Dropout
from keras.layers import *
from keras.callbacks import EarlyStopping
import pydot as pyd
from IPython.display import SVG
from keras.utils.vis_utils import plot_model #plot network's structure
import matplotlib.pyplot as plt #plots
import time
from collections import Counter
import os
os.chdir('..')
import csv #save output
import math

# IMPORT DATA
Import data and plot the variables

In [ ]:
data_US_gm_12= pd.read_csv(r'C:/Users/rneb3bv/Desktop/Inflation_Trade_Thesis/US_Monthly(gm)_csv2.csv',  sep = ',', index_col = 0)

In [ ]:
data_US_gm_12.plot(kind='line',
                subplots=True,
                grid=True,
                figsize=(12, 12),
                title=['Core CPI(yy)', 'Oil(eur_yy)', 'Output Gap', 'NEER(yy)', 'World CPI (yy)', 'US Michigan 5y'],
                layout=(3, 2),
                sharex=True,
                sharey=False,
                legend=False,
                style=['darkred', 'steelblue', 'dodgerblue', 'slateblue', 'darkslateblue', 'mediumblue',
                       'midnightblue'])

[ax.set_xlabel('') for ax in plt.gcf().axes]
plt.suptitle('Data for the United States', fontsize=20)
plt.show()

# Neural Network for the US
The first step is to split data into training and test sets. I will be using data from the beginning until 2009 for training and from 2009 to 2015 for testing. I will leave the last 12 observations for out-of-sample testing later. The testing inputs are data from 2014-2015. 
PS:To construct inflation surprise rate I started from the period of 04/2022-03/2023 and then forecasted previous 12 months and so forth until 01/1982. Here, we have prediction codes for period of 2014-2015.
The following function splits a multivariate sequence into samples:

In [ ]:
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()    
    for i in range(len(sequences)):
        #find end of pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1
        
        #check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        
        #gather input and output
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix - 1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)

In [ ]:

data_US_gm_12.loc['15/04/2004':'15/03/2014', ['CPI']]  #in-sample data
data_US_gm_12.loc['15/04/2014':'15/03/2015', ['CPI']]  #out-of-sample target

In [ ]:
#inputs
in_core_cpi_12 = array(data_US_gm_12.loc['15/01/1982':'15/01/2009', ['CPI']])
in_outgap_12 = array(data_US_gm_12.loc['15/01/1982':'15/01/2009', ['Output_gap']])
in_NEERate_12  = array(data_US_gm_12.loc['15/01/1982':'15/01/2009', ['NEER']])
in_world_cpi_12 = array(data_US_gm_12.loc['15/01/1982':'15/01/2009', ['World_cpi']])
in_oil_e_12   = array(data_US_gm_12.loc['15/01/1982':'15/01/2009', ['Crude oil']])
in_UMic5y_12   = array(data_US_gm_12.loc['15/01/1982':'15/01/2009', ['US_Michigan_5y']])

valin_core_cpi_12 = array(data_US_gm_12.loc['15/01/2009':'15/03/2015', ['CPI']])
valin_outgap_12 = array(data_US_gm_12.loc['15/01/2009':'15/03/2015', ['Output_gap']])
valin_NEERate_12  = array(data_US_gm_12.loc['15/01/2009':'15/03/2015', ['NEER']])
valin_world_cpi_12 = array(data_US_gm_12.loc['15/01/2009':'15/03/2015', ['World_cpi']])
valin_oil_e_12  = array(data_US_gm_12.loc['15/01/2009':'15/03/2015', ['Crude oil']])
valin_UMic5y_12   = array(data_US_gm_12.loc['15/01/2009':'15/03/2015', ['US_Michigan_5y']])

#output
out_infl_12 = array(data_US_gm_12.loc['15/01/1982':'15/01/2009', ['CPI']])

valout_infl_12 = array(data_US_gm_12.loc['15/01/2009':'15/03/2015', ['CPI']])

#reshape to [rows, columns]
in_core_cpi_12 = in_core_cpi_12.reshape((len(in_core_cpi_12), 1))
in_outgap_12 = in_outgap_12.reshape((len(in_outgap_12), 1))
in_NEERate_12  = in_NEERate_12.reshape((len(in_NEERate_12), 1))
in_world_cpi_12 = in_world_cpi_12.reshape((len(in_world_cpi_12), 1))
in_oil_e_12   = in_oil_e_12.reshape((len(in_oil_e_12), 1))
in_UMic5y_12 = in_UMic5y_12.reshape((len(in_UMic5y_12), 1))

valin_core_cpi_12 = valin_core_cpi_12.reshape((len(valin_core_cpi_12), 1))
valin_outgap_12 = valin_outgap_12.reshape((len(valin_outgap_12), 1))
valin_NEERate_12  = valin_NEERate_12.reshape((len(valin_NEERate_12), 1))
valin_world_cpi_12 = valin_world_cpi_12.reshape((len(valin_world_cpi_12), 1))
valin_oil_e_12   = valin_oil_e_12.reshape((len(valin_oil_e_12), 1))
valin_UMic5y_12= valin_UMic5y_12.reshape((len(valin_UMic5y_12), 1))


out_infl_12 = out_infl_12.reshape((len(out_infl_12), 1))
valout_infl_12 = valout_infl_12.reshape((len(valout_infl_12), 1))

#stack columns horizontally
dataset_12 = hstack((in_core_cpi_12, in_outgap_12, in_NEERate_12, in_world_cpi_12, in_oil_e_12, in_UMic5y_12, out_infl_12))
valset_12= hstack((valin_core_cpi_12, valin_outgap_12, valin_NEERate_12, valin_world_cpi_12, valin_oil_e_12, valin_UMic5y_12, valout_infl_12))




#timesteps
n_steps_in, n_steps_out = 12,12 #use 6 observations to predict next 6

X_12, y_12 = split_sequences(dataset_12, n_steps_in, n_steps_out)
print(X_12.shape, y_12.shape) #shape of input and output = (21, 12, 6) (21, 12)
valX_12, valy_12 = split_sequences(valset_12, n_steps_in, n_steps_out)

# Note that I have prepared the data, we can continue with the construction of the model. Lets start by doing an LSTM for multi-step predictions.

In [ ]:
x_core_cpi_12 = array(data_US_gm_12.loc['15/04/2014':'15/03/2015', ['CPI']])
x_outgap_12 = array(data_US_gm_12.loc['15/04/2014':'15/03/2015', ['Output_gap']])
x_NEERate_12  = array(data_US_gm_12.loc['15/04/2014':'15/03/2015', ['NEER']])
x_world_cpi_12 = array(data_US_gm_12.loc['15/04/2014':'15/03/2015', ['World_cpi']])
x_oil_e_12  = array(data_US_gm_12.loc['15/04/2014':'15/03/2015', ['Crude oil']])
x_UMic5y_12 = array(data_US_gm_12.loc['15/04/2014':'15/03/2015', ['US_Michigan_5y']])

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Define the range of hyperparameters to search
param_range = {
    'n_units': [50, 100],
    'dropout': [0.2, 0.4],
    'recurrent_dropout': [0.2, 0.4],
    'learning_rate': [0.0001, 0.001]
}

best_loss = float('inf')
best_model = None

n_features = X_12.shape[2]

for n_units in param_range['n_units']:
    for dropout in param_range['dropout']:
        for recurrent_dropout in param_range['recurrent_dropout']:
            for learning_rate in param_range['learning_rate']:
                # Build the model
                model = Sequential()
                model.add(LSTM(n_units,
                               activation='tanh',
                               return_sequences=True,
                               input_shape=(n_steps_in, n_features)))
                model.add(LSTM(n_units,
                               dropout=dropout,
                               recurrent_dropout=recurrent_dropout,
                               activation='tanh'))
                model.add(Dense(50, activation='tanh'))
                model.add(Dense(25, activation='tanh'))
                model.add(Dense(n_steps_out))

                # Compile the model
                model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                              loss='mse')

                # Define early stopping
                es = EarlyStopping(monitor='val_loss',
                                   mode='min',
                                   min_delta=0.01,
                                   patience=30,
                                   verbose=0)

                # Fit the model
                fit = model.fit(X_12,
                                y_12,
                                validation_data=(valX_12, valy_12,
                                epochs=500,
                                verbose=0,
                                callbacks=[es])

                # Evaluate the model
                val_loss = np.min(fit.history['val_loss'])

                # Check if this model has the best loss so far
                if val_loss < best_loss:
                    best_loss = val_loss
                    best_model = model

#prediction
x_input_12 = array([x_core_cpi_12,
                 x_outgap_12,
                 x_NEERate_12,
                 x_world_cpi_12,
                 x_oil_e_12,
                 x_UMic5y_12])
x_input_12 = x_input_12.reshape((1, n_steps_in, n_features))

yhat_12 = best_model.predict(x_input_12, verbose=0)
print(yhat_12)


# Learning curve 

In [ ]:
plt.plot(fit.history['loss'], label='training', color='SteelBlue')
plt.plot(fit.history['val_loss'], label='validation', color='DarkRed')
plt.legend()
plt.show


In [ ]:
predicted_12 = pd.DataFrame(yhat_12.T, index=pd.date_range('15/03/2020',periods=12,freq='M'))
predicted_12.columns = ['predicted']

predicted_12.plot(color = 'SteelBlue')
plt.show()



observed_12 = data_US_gm_12.loc['15/04/2020':'15/03/2021', ['CPI']]
observed_12.columns = ['observed']

observed_12.plot(color = 'DarkRed')
plt.show()

model_error_12 = array(predicted_12) - array(observed_12)
model_error_12 = pd.DataFrame(model_error_12, index=pd.date_range('15/03/2020',periods=12,freq='M'))
model_error_12.columns = ['forecast error']

model_error_12.plot(color = 'goldenrod')
plt.show()

In [ ]:
model_error_12.mean()

# Although the LSTM is able to approximate the 'shape' of future inflation, there is still room for improvement. forecast error is around 0.425431 (I haven't set a seed yet, so it may change), implying that the model overestimates inflation (remember that instead of y-yhat , I'm using yht-y, allowing for a more direct interpretation).

# Now, we build a prediction iterval for the network. I train the network 50 times with a for loop and store the results in the results variable
PS: Ranges for the hyperparameters are selected according to the pattern of each , figured by running the model again and again

In [ ]:
n = 50
results = []
for i in range(0, n):
# Define the range of hyperparameters to search
    param_range = {
        'n_units': [50, 100],
        'dropout': [0.2, 0.4],
        'recurrent_dropout': [0.2, 0.4],
        'learning_rate': [0.0001, 0.001]
    }

    best_loss = float('inf')
    best_model = None

    n_features = X_12.shape[2]

    for n_units in param_range['n_units']:
        for dropout in param_range['dropout']:
            for recurrent_dropout in param_range['recurrent_dropout']:
                for learning_rate in param_range['learning_rate']:
                    # Build the model
                    model = Sequential()
                    model.add(LSTM(n_units,
                                   activation='tanh',
                                   return_sequences=True,
                                   input_shape=(n_steps_in, n_features)))
                    model.add(LSTM(n_units,
                                   dropout=dropout,
                                   recurrent_dropout=recurrent_dropout,
                                   activation='tanh'))
                    model.add(Dense(50, activation='tanh'))
                    model.add(Dense(25, activation='tanh'))
                    model.add(Dense(n_steps_out))

                    # Compile the model
                    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                                  loss='mse')

                    # Define early stopping
                    es = EarlyStopping(monitor='val_loss',
                                       mode='min',
                                       min_delta=0.01,
                                       patience=30,
                                       verbose=0)

                    # Fit the model
                    fit = model.fit(X_12,
                                    y_12,
                                    validation_data=(valX_12, valy_12),
                                    epochs=500,
                                    verbose=0,
                                    callbacks=[es])

                    # Evaluate the model
                    val_loss = np.min(fit.history['val_loss'])

                    # Check if this model has the best loss so far
                    if val_loss < best_loss:
                        best_loss = val_loss
                        best_model = model

    #prediction
    x_input_12 = array([x_core_cpi_12,
                     x_outgap_12,
                     x_NEERate_12,
                     x_world_cpi_12,
                     x_oil_e_12,
                     x_UMic5y_12])
    x_input_12 = x_input_12.reshape((1, n_steps_in, n_features))

    yhat_12 = best_model.predict(x_input_12, verbose=0)
    results.append((yhat_12))

In [ ]:
iter_output = pd.DataFrame(np.concatenate(results)) #build data frame with results
iter_output.columns = ['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12']
iter_output

# I receive 50 forecast values for each time period i.e. m1 , m2 etc. and take the mean of them to get one forecast value.


In [ ]:
mean_forecast = pd.DataFrame([iter_output['m1'].mean(),
                              iter_output['m2'].mean(),
                              iter_output['m3'].mean(),
                              iter_output['m4'].mean(),
                              iter_output['m5'].mean(),
                              iter_output['m6'].mean(),
                              iter_output['m7'].mean(),
                              iter_output['m8'].mean(),
                              iter_output['m9'].mean(),
                              iter_output['m10'].mean(),
                              iter_output['m11'].mean(),
                              iter_output['m12'].mean()],
                             index=pd.date_range('15/04/2019', periods=12, freq='M'))  #mean dataframe
mean_forecast.columns = ['predicted']
mean_forecast


# Standard Errors

In [ ]:
se_forecast = pd.DataFrame([iter_output['m1'].std(),
                            iter_output['m2'].mean(),
                            iter_output['m3'].mean(),
                            iter_output['m4'].mean(),
                            iter_output['m5'].mean(),
                            iter_output['m6'].mean(),
                            iter_output['m7'].mean(),
                            iter_output['m8'].mean(),
                            iter_output['m9'].mean(),
                            iter_output['m10'].mean(),
                            iter_output['m11'].mean(),
                            iter_output['m12'].mean()],
                           index=pd.date_range('15/04/2019', periods=12, freq='m'))  #se dataframe
se_forecast.columns = ['predicted']
se_forecast

# Plot confidence intervals , predictions

In [ ]:
upper_bound = mean_forecast + 2 * se_forecast
lower_bound = mean_forecast - 2 * se_forecast
plt.plot(mean_forecast, label='predicted', color='SteelBlue')
plt.plot(upper_bound, label='95% confidence interval', color='SkyBlue')
plt.plot(lower_bound, color='SkyBlue')
plt.legend()
plt.show



# Observed values

In [ ]:
observed = pd.DataFrame(pd.read_csv(r'C:/Users/rneb3bv/Desktop/Inflation_Trade_Thesis/US_Monthly(gm)_csv2.csv',
                                    sep=',',
                                    skiprows= 447,
                                    skipfooter=36,
                                    usecols=[1]
                                    ))

#observed = observed.dropna()
observed


In [ ]:
observed.columns = ['observed']
observed.index = pd.date_range('15/04/2019',periods=12,freq='M')
observed


# Plot for actual values vs predictions

In [ ]:
observed
plt.plot(mean_forecast, label='predicted', color='SteelBlue')
plt.plot(upper_bound, label='95% confidence interval', color='lightblue')
plt.plot(lower_bound, color='lightblue')
plt.plot(observed, label='observed', color='DarkRed')
plt.legend()
plt.show

model_error = array(mean_forecast) - array(observed)
model_error = pd.DataFrame(model_error, index=pd.date_range('15/04/2019', periods=12, freq='M'))
model_error.columns = ['forecast error']

model_error.plot(color='goldenrod')
plt.show()

model_error.mean(), mean_forecast.mean()
